# Методы сбора и обработки данных из сети Интернет
## Урок 4. Система управления базами данных MongoDB в Python
1. Развернуть у себя на компьютере/виртуальной машине/хостинге MongoDB и реализовать функцию, которая будет добавлять только новые вакансии/продукты в вашу базу.
2. Написать функцию, которая производит поиск и выводит на экран вакансии с заработной платой больше введённой суммы (необходимо анализировать оба поля зарплаты). Для тех, кто выполнил задание с Росконтролем - напишите запрос для поиска продуктов с рейтингом не ниже введенного или качеством не ниже введенного (то есть цифра вводится одна, а запрос проверяет оба поля).
Сдать в виде ссылки на репозиторий.
Задание считается выполненным, если:
- код задокументирован, парсер работает корректно
- репозиторий имеет readme.md с описанием и структурой проекта
- есть скриншоты результата сбора данных и обращения к БД

### Установлю mongo 

In [13]:
pip install pymongo

Note: you may need to restart the kernel to use updated packages.


In [14]:
from pprint import pprint
from bs4 import BeautifulSoup as bs
import requests
from time import sleep
from pymongo import MongoClient

### Спарсим данные

In [31]:
name = input('Введите название професии')
url = 'https://hh.ru/search/vacancy?text=' + name
print(url)
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 YaBrowser/23.1.2.928 Yowser/2.5 Safari/537.36',
}
response = requests.get(url=url, headers=headers)
response.text

Введите название професииData science стажёр
https://hh.ru/search/vacancy?text=Data science стажёр


'<!DOCTYPE html>\n<html class="desktop" lang="ru"><!--request: 1676897948496f5a2a6e3bf2bfcd1cd8--><head><meta http-equiv="Content-Type" content="text/html; charset=UTF-8"><meta http-equiv="X-UA-Compatible" content="IE=edge"><title data-rh="" data-reactroot="">Работа в Волжском (Волгоградская область), поиск персонала и публикация вакансий - volzhskiy.hh.ru</title><meta name="twitter:card" content="summary_large_image"><meta data-rh="" name="description" content="volzhskiy.hh.ru — сервис, который помогает найти работу и подобрать персонал в Волжском (Волгоградская область) более 20 лет! Создавайте резюме и откликайтесь на вакансии. Набирайте сотрудников и публикуйте вакансии." data-reactroot=""><meta data-rh="" name="keywords" content="работа, вакансии, работа, поиск вакансий, резюме, работы, работу, работ, ищу работу, поиск" data-reactroot=""><meta property="og:image" content="https://i.hh.ru/logos/openGraph/hh.ru.png?v=12012023?v=3"><meta property="og:image:width" content="600"><meta 

In [33]:
def parse_hh(url_page, headers, result=[], index_page=1):
    response = requests.get(url_page, headers=headers)
    if response.status_code != 200:
      print('Парсинг завершен')
      return result
    else:
      print('Cтраница №%d, ссылка: %s'%(index_page, response.url))
    
    dom = bs(response.content, 'html.parser')
    vacancies = dom.find_all('div', {'class': 'vacancy-serp-item__layout'})
    for vacancy in vacancies:
      result.append(parse_vacancy_hh(vacancy))

    link_next_page = dom.find('a', {'data-qa': 'pager-next'})
    if link_next_page:
      link_next_page =  'https://spb.hh.ru' + link_next_page['href']
    else:
      print('Парсинг завершен')
      return result

    result = parse_hh(link_next_page, headers, result, index_page+1)   
    return result
     
def parse_vacancy_hh(dom_vacancy):
  sleep(0.2)
  vacancy_name = dom_vacancy.find('a').text
  vacancy_salary = dom_vacancy.find('span', {'class', 'bloko-header-section-3'})
  if vacancy_salary:
    vacancy_salary = vacancy_salary.text
    min_salary, max_salary, currency_salary = clean_salary(vacancy_salary)
  
  else:
    min_salary, max_salary, currency_salary = None, None, None

  vacancy_link = dom_vacancy.find('a')['href']
  
  return {
      'vacancy_name': vacancy_name,
      'vacancy_salary': vacancy_salary,
      'min_salary': min_salary,
      'max_salary': max_salary,
      'currency_salary': currency_salary,
      'vacancy_link': vacancy_link,
      'vacancy_source': 'hh.ru',
  }

def clean_salary(vacancy_salary_text, min_salary=None, max_salary=None, currency_salary=None):
  list_salary = vacancy_salary_text.replace('\u202f', '').split()
  for i in range(len(list_salary) - 1):
    if list_salary[i] == 'от':
      min_salary = int(list_salary[i + 1])
    elif list_salary[i] == 'до':
      max_salary = int(list_salary[i + 1])
    elif list_salary[i] == '–':
      min_salary = int(list_salary[i - 1])
      max_salary = int(list_salary[i + 1])
  currency_salary = list_salary[-1]

  return min_salary, max_salary, currency_salary

In [34]:
result = parse_hh(url, headers)

Cтраница №1, ссылка: https://volzhskiy.hh.ru/search/vacancy?text=Data%20science%20%D1%81%D1%82%D0%B0%D0%B6%D1%91%D1%80
Cтраница №2, ссылка: https://spb.hh.ru/search/vacancy?text=Data+science+%D1%81%D1%82%D0%B0%D0%B6%D1%91%D1%80&page=1&hhtmFrom=vacancy_search_list
Cтраница №3, ссылка: https://spb.hh.ru/search/vacancy?text=Data+science+%D1%81%D1%82%D0%B0%D0%B6%D1%91%D1%80&page=2&hhtmFrom=vacancy_search_list
Парсинг завершен


In [35]:
len(result)

60

In [44]:
client = MongoClient()

In [45]:
db = client.vacancies_hh

In [46]:
collection_vacancies_hh_ru = db.hh_ru

In [47]:
def cheak_and_save_vacancies_in_db(vacancies):
  for vacancy in vacancies:
    if not len(list(collection_vacancies_hh_ru.find({'vacancy_link': vacancy['vacancy_link']}))):
      collection_vacancies_hh_ru.insert_one(vacancy)

In [48]:
cheak_and_save_vacancies_in_db(result)
result_find = list(collection_vacancies_hh_ru.find())
len(result_find)

60